# Single Container Model Development

In this notebook, our goal is to develop the necessary model for a single container
with varying concurrency levels.


In [1]:
%load_ext autoreload
%autoreload 2
# imports

# important libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import scipy as sp

from tqdm.auto import tqdm

# for better printing of variables
from IPython.display import display

In [2]:
# custom imports
from concperf import single_container_model as single_model

In [3]:
# update configuration dictionary
def update_config(config):
    config['arrival_rate_server'] = config['arrival_rate_total'] / config['instance_count']
    config['base_service_time'] = config['base_service_time_ms'] / 1000

# calculate Q for states
def get_single_container_q(single_coder, config):
    state_count = single_coder.get_state_count()
    Q = np.zeros((state_count, state_count))

    def encode_state(req_count):
        return (req_count,)

    # for each source state calculate rate of destination state
    for from_state in single_coder.get_state_list():
        from_state_idx = single_coder.to_idx(from_state)
        # decode the state
        (from_req_count, ) = from_state
        # rate of exiting this state
        exit_rate = 0
        # one count below
        if from_req_count > 0:
            to_req_count = from_req_count - 1
            to_state = encode_state(to_req_count)
            to_state_idx = single_coder.to_idx(to_state)
            Q[from_state_idx, to_state_idx] = from_req_count / (1+(from_req_count - 1)*config['alpha']) / config['base_service_time']
            exit_rate += Q[from_state_idx, to_state_idx]
        if from_req_count < config['max_conc']:
            to_req_count = from_req_count + 1
            to_state = encode_state(to_req_count)
            to_state_idx = single_coder.to_idx(to_state)
            Q[from_state_idx, to_state_idx] = config['arrival_rate_server']
            exit_rate += Q[from_state_idx, to_state_idx]
        
        Q[from_state_idx, from_state_idx] = -1 * exit_rate

    return Q

def solve_CTMC(Q):
    # solve CTMC for pi
    state_count = Q.shape[0]
    Q[:, 0] = 1
    y = np.zeros((1, Q.shape[0]))
    y[0, 0] = 1
    solution = np.linalg.solve(np.array(Q.T), np.array(y.T))
    solution = solution.reshape(solution.shape[0],)
    solution[solution < 0] = 0
    return solution


######################################################
# using the defined functions

single_container_config = {
    'instance_count': 10,
    'max_conc': 10,
    'arrival_rate_total': 5,
    'alpha': 0.11,
    'base_service_time_ms': 1154,
}

# update the config
update_config(single_container_config)
display(single_container_config)

single_coder = single_model.StateCoder(config=single_container_config)

# calculate and show Q
Q = get_single_container_q(single_coder, config=single_container_config)
display(pd.DataFrame(Q))

req_count_prob = solve_CTMC(Q)
# display solution for debug purposes
display([f"{s:4.2f}" for s in req_count_prob])

# calculate average and deviation
req_df = pd.DataFrame(data = {
    'req_count': [s[0] for s in single_coder.get_state_list()],
    'req_count_prob': req_count_prob,
})
display(req_df)

# calculate average concurrency
req_count_avg = (req_df['req_count'] * req_df['req_count_prob']).sum()
print(f"Concurrency: {req_count_avg}(avg) +- ?(std)")

{'instance_count': 10,
 'max_conc': 10,
 'arrival_rate_total': 5,
 'alpha': 0.11,
 'base_service_time_ms': 1154,
 'arrival_rate_server': 0.5,
 'base_service_time': 1.154}

,0,1,2,3,4,5,6,7,8,9,10
0,-0.500000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.866551,-1.366551,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,1.561353,-2.061353,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,2.130863,-2.630863,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,2.606169,-3.106169,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,3.008858,-3.508858,0.500000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,3.354391,-3.854391,0.500000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.654131,-4.154131,0.500000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.916615,-4.416615,0.500000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.148383,-4.648383,0.500000


['0.55',
 '0.32',
 '0.10',
 '0.02',
 '0.00',
 '0.00',
 '0.00',
 '0.00',
 '0.00',
 '0.00',
 '0.00']

,req_count,req_count_prob
0,0,5.509425e-01
1,1,3.178938e-01
2,2,1.018007e-01
3,3,2.388720e-02
4,4,4.582820e-03
5,5,7.615546e-04
6,6,1.135161e-04
7,7,1.553257e-05
8,8,1.982907e-06
9,9,2.389975e-07


Concurrency: 0.6161040935667044(avg) +- ?(std)
